In [ ]:
import mlflow
import optuna
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
from lightgbm import LGBMClassifier
from imblearn.over_sampling import ADASYN
from imblearn.pipeline import make_pipeline

# --- 1. Load & Process Data ---
try:
    df = pd.read_csv('reddit_preprocessed.csv')
except FileNotFoundError:
    print("❌ Error: 'reddit_preprocessed.csv' not found. Please upload your dataset.")
    raise

df = df.drop('Unnamed: 0', axis=1, errors='ignore')
df = df.dropna()
df['clean_comment'] = df['clean_comment'].astype(str)

le = LabelEncoder()
y = le.fit_transform(df['category'])
class_names = [str(cls) for cls in le.classes_] # e.g. ['Negative', 'Neutral', 'Positive']

# ✅ FIX LEAKAGE: Split Raw Text FIRST
X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    df['clean_comment'], y, test_size=0.2, random_state=42, stratify=y
)

# --- 2. Optuna Tuning (CPU Mode) ---
mlflow.set_experiment("EXP 5 - ML Algos with optuna")
optuna.logging.set_verbosity(optuna.logging.WARNING)

def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),
        'verbose': -1,
        'n_jobs': 1  # 1 job per trial prevents CPU contention
    }

    # Pipeline: Vectorizer -> ADASYN -> LGBM
    # No "to_dense" needed for CPU (LightGBM CPU handles sparse matrices fine)
    clf = make_pipeline(
        TfidfVectorizer(max_features=3000, ngram_range=(1, 2)),
        ADASYN(random_state=42),
        LGBMClassifier(**params, random_state=42)
    )

    # Cross Val on raw text
    return cross_val_score(clf, X_train_raw, y_train, cv=3, scoring='accuracy').mean()

print("🚀 Tuning LightGBM with ADASYN (CPU Mode)...")
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

# --- 3. Final Training ---
print("✅ Tuning complete. Training final model...")
best_params = study.best_params

final_pipeline = make_pipeline(
    TfidfVectorizer(max_features=3000, ngram_range=(1, 2)),
    ADASYN(random_state=42),
    LGBMClassifier(**best_params, random_state=42, verbose=-1, n_jobs=-1) # n_jobs=-1 uses all CPU cores
)

final_pipeline.fit(X_train_raw, y_train)
y_pred = final_pipeline.predict(X_test_raw)

# --- 4. Detailed Logging ---
with mlflow.start_run(run_name="LGBM_ADASYN_CPU_Detailed"):
    # Log Parameters
    mlflow.log_params(best_params)
    mlflow.log_param("features", 3000)
    mlflow.log_param("ngram", "1,2")

    # Generate Report (output_dict=True for parsing)
    report_dict = classification_report(y_test, y_pred, target_names=class_names, output_dict=True)

    # Print readable report to console
    print("\n📊 Detailed Classification Report:")
    print(classification_report(y_test, y_pred, target_names=class_names))

    # Log Global Metrics
    mlflow.log_metric("accuracy", report_dict['accuracy'])
    mlflow.log_metric("macro_f1", report_dict['macro avg']['f1-score'])

    # Log Class-Specific Metrics (Precision, Recall, F1 for every class)
    # Loop through the dictionary keys (e.g., 'Negative', 'Positive', etc.)
    for label in class_names:
        if label in report_dict:
            metrics = report_dict[label]
            mlflow.log_metric(f"{label}_precision", metrics['precision'])
            mlflow.log_metric(f"{label}_recall", metrics['recall'])
            mlflow.log_metric(f"{label}_f1", metrics['f1-score'])

    print(f"✅ MLflow Run Complete. Accuracy: {report_dict['accuracy']:.4f}")

🚀 Tuning LightGBM with ADASYN (CPU Mode)...


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/ut

✅ Tuning complete. Training final model...


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(



📊 Detailed Classification Report:
              precision    recall  f1-score   support

          -1       0.80      0.73      0.76      1650
           0       0.85      0.96      0.90      2529
           1       0.89      0.84      0.87      3154

    accuracy                           0.86      7333
   macro avg       0.85      0.84      0.84      7333
weighted avg       0.86      0.86      0.85      7333

✅ MLflow Run Complete. Accuracy: 0.8565
🏃 View run LGBM_ADASYN_CPU_Detailed at: https://dagshub.com/Ritk-Raikwar/reddit-comment-sentiment-analysis.mlflow/#/experiments/5/runs/32f1a24adb0941ccbe164e1b26ca3054
🧪 View experiment at: https://dagshub.com/Ritk-Raikwar/reddit-comment-sentiment-analysis.mlflow/#/experiments/5


In [ ]:
!pip install mlflow dagshub optuna imbalanced-learn lightgbm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 89.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 90.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.0/262.0 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.9/413.9 kB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.2/131.2 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/

In [ ]:
import dagshub
dagshub.init(repo_owner='Ritk-Raikwar', repo_name='reddit-comment-sentiment-analysis', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=d1ea97d7-c4e7-4e98-8e7f-c88b5dd10cd5&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=42053c97ed658add795f87b6bf53776f1b6ebdf79ef96ddd2a988115b0bc760b




Accessing as Ritk-Raikwar

Initialized MLflow to track repo "Ritk-Raikwar/reddit-comment-sentiment-analysis"

Repository Ritk-Raikwar/reddit-comment-sentiment-analysis initialized!

In [ ]:
# 2. Install GPU version (Standard T4/L4/A100 GPU)
# We force the build to use CUDA/OpenCL
# !pip install lightgbm \
#     --no-binary lightgbm \
#     --config-settings=cmake.define.USE_GPU=ON \
#     --config-settings=cmake.define.OpenCL_INCLUDE_DIR="/usr/local/cuda/include/" \
#     --config-settings=cmake.define.OpenCL_LIBRARY="/usr/local/cuda/lib64/libOpenCL.so"

# print("✅ LightGBM GPU version installed. Please RESTART YOUR RUNTIME (Runtime > Restart Runtime).")

✅ LightGBM GPU version installed. Please RESTART YOUR RUNTIME (Runtime > Restart Runtime).
